<a href="https://colab.research.google.com/github/vandana-iyer/RecSys-Instacart/blob/master/InstacartRecsysAlgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Authenticate and download datasets from Kaggle

!mkdir -p ~/.kaggle
!wget -O ~/.kaggle/kaggle.json https://www.dropbox.com/s/b6g4cawc664i4u9/kaggle.json?dl=0
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

!pip install --upgrade kaggle

!kaggle competitions download -c instacart-market-basket-analysis
!unzip instacart-market-basket-analysis.zip && rm instacart-market-basket-analysis.zip

# unzip all csv files
!unzip \*.csv.zip
# remove unwanted files/directories
!rm -rf __MACOSX
# !rm *.zip

--2020-04-24 05:06:39--  https://www.dropbox.com/s/b6g4cawc664i4u9/kaggle.json?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/b6g4cawc664i4u9/kaggle.json [following]
--2020-04-24 05:06:39--  https://www.dropbox.com/s/raw/b6g4cawc664i4u9/kaggle.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc66f933936ba79ddc324c5b1143.dl.dropboxusercontent.com/cd/0/inline/A2YrFkzXg1E4cOmn31-y3sHCVf85LN48jz0RXKq8lZ6Q2hS9ywElNRWD0tzx-HqMs1uCqMqVX11QCXPalu5bThjOuCWVmAwH5rVwRUmUpquw87630zN8lOUehMoQqlyiV1Q/file# [following]
--2020-04-24 05:06:40--  https://uc66f933936ba79ddc324c5b1143.dl.dropboxusercontent.com/cd/0/inline/A2YrFkzXg1E4cOmn31-y3sHCVf85LN48jz0RXKq8lZ6Q2hS9ywElNRWD0tzx-HqMs1uCqMqVX11QCXPalu5bThjOuCWVmAwH5rVwRUmUpquw8

In [3]:
!pip install surprise

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')
order_products = pd.read_csv('order_products__train.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')

In [6]:
# Merge products, aisles and departments
products_dept_aisles_df = products.merge(aisles, on='aisle_id').merge(departments, on='department_id')
products_dept_aisles_df.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks


In [7]:
# Merge order and product_orders

print(order_products.shape)
print(order_products_prior.shape)
print(orders.shape)

user_product_orders = order_products.append(order_products_prior).merge(orders, on='order_id')
user_product_orders.shape

(1384617, 4)
(32434489, 4)
(3421083, 7)


(33819106, 10)

In [8]:
# Merge all tables. Create train dataset

data = user_product_orders.merge(products_dept_aisles_df, on='product_id')
data.shape

(33819106, 15)

In [9]:
%%time
import numpy
import numpy as np
import pandas as pd
import progressbar as pb

def matrix_factorization(R, P, Q, K, steps=1000, alpha=0.0002, beta=0.02, tol=0.001):
    Q = np.transpose(Q)
    non_zero_indexes = np.nonzero(R)
    for step in pb.progressbar(range(steps)):
      for i, j in zip(non_zero_indexes[0], non_zero_indexes[1]):
        eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
        P[i, :] += alpha * (2 * eij * np.transpose(Q[:, j]) - beta * P[i, :])
        Q[:, j] += alpha * (2 * eij * np.transpose(P[i, :]) - beta * Q[:, j])

      e = 0
      for i, j in zip(non_zero_indexes[0], non_zero_indexes[1]):
        e += pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
        e += (beta/2) * np.sum(pow(P[i,:],2) + pow(Q[:,j],2))

      if e < 0.001:
        break

    return P, Q.T

R = [
     [5,3,0,1],
     [4,0,0,1],
     [1,1,0,5],
     [1,0,0,4],
     [0,1,5,4],
    ]

R = numpy.array(R)
N = len(R)
M = len(R[0])
K = 2

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)
nR = numpy.dot(nP, nQ.T)
print(nR)

100% (1000 of 1000) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


[[3.95952805 1.92838403 3.73287715 2.21125506]
 [2.45772123 1.1970826  2.76736337 2.15322238]
 [2.19880928 1.0712544  3.58128196 3.84276325]
 [2.04801863 0.99768735 2.93223127 2.87982017]
 [3.73729717 1.82044013 4.6593251  4.05640924]]
CPU times: user 518 ms, sys: 14.7 ms, total: 533 ms
Wall time: 535 ms


#### User Department embeddings

In [10]:
%%time
# Dept User embeddings

data_user_department = data.groupby(['user_id', 'department_id']).agg({'product_id': 'count'}).rename(columns={'product_id': 'total_products'})

# Alternatively, use pivot_table as shown below, but is slower by few milliseconds seconds as compared to above groupby + unstack()
# data.pivot_table(values='product_id', index='user_id', columns='department_id', aggfunc='count')

CPU times: user 5.92 s, sys: 2.34 s, total: 8.26 s
Wall time: 8.26 s


In [0]:
np_array_data_user_department = data_user_department.unstack('department_id').values

In [0]:
user_dept_surprise_data = data_user_department.reset_index()

In [12]:
np_array_data_user_department

array([[nan, nan, nan, ..., 25., nan, nan],
       [28., nan,  2., ..., 47., 24., nan],
       [ 6., nan, nan, ...,  9.,  2., nan],
       ...,
       [21., nan,  3., ..., 23.,  8., nan],
       [26., nan, 62., ..., 59., 28.,  2.],
       [ 3., nan, 10., ..., 12.,  6., nan]])

In [13]:
np_array_data_user_department = np.nan_to_num(np_array_data_user_department).astype(int)
np_array_data_user_department

array([[ 0,  0,  0, ..., 25,  0,  0],
       [28,  0,  2, ..., 47, 24,  0],
       [ 6,  0,  0, ...,  9,  2,  0],
       ...,
       [21,  0,  3, ..., 23,  8,  0],
       [26,  0, 62, ..., 59, 28,  2],
       [ 3,  0, 10, ..., 12,  6,  0]])

In [0]:
# R = np_array_data_user_department

# N = len(R)
# M = len(R[0])
# K = 2

# P = numpy.random.rand(N,K)
# Q = numpy.random.rand(M,K)

# nP, nQ = matrix_factorization(R, P, Q, K)
# nR = numpy.dot(nP, nQ.T)

# nR

User Aisle embeddings

In [19]:
%%time

# User Aisle embeddings

user_aisle = data.groupby(['user_id', 'aisle_id']).agg({'product_id': 'count'}).rename(columns={'product_id': 'total_products'})
np_array_user_aisle = user_aisle.unstack('aisle_id').values
np_array_user_aisle = np.nan_to_num(np_array_user_aisle).astype(int)

CPU times: user 6.81 s, sys: 16.9 ms, total: 6.83 s
Wall time: 6.83 s


In [33]:
user_id = user_aisle.index.get_level_values(level=0)
aisle_id = 

Int64Index([     1,      1,      1,      1,      1,      1,      1,      1,
                 1,      1,
            ...
            206209, 206209, 206209, 206209, 206209, 206209, 206209, 206209,
            206209, 206209],
           dtype='int64', name='user_id', length=5919840)

In [30]:
%%time

from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import PredefinedKFold

reader = Reader()

test_data = data[:]
test_data['product_id'] = np.log(test_data['product_id'] )

surprise_data = Dataset.load_from_df(user_dept_surprise_data, reader)

CPU times: user 5.05 s, sys: 256 ms, total: 5.31 s
Wall time: 5.31 s


In [31]:
%%time

from surprise.model_selection import cross_validate

algo = SVD()
cross_validate(algo, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    33.2584 33.3759 33.2552 33.0296 33.5212 33.2881 0.1618  
MAE (testset)     12.3803 12.4570 12.4603 12.4093 12.4567 12.4327 0.0323  
Fit time          110.10  119.60  111.11  119.54  112.33  114.54  4.17    
Test time         5.33    5.96    5.29    5.30    6.00    5.57    0.33    


{'fit_time': (110.10284519195557,
  119.60441756248474,
  111.1067008972168,
  119.54339289665222,
  112.33499431610107),
 'test_mae': array([12.38032488, 12.45699218, 12.46028657, 12.40932992, 12.45667755]),
 'test_rmse': array([33.25842301, 33.37589921, 33.25519773, 33.02955552, 33.52122754]),
 'test_time': (5.330892324447632,
  5.957203388214111,
  5.285067558288574,
  5.296467542648315,
  5.999311685562134)}

In [32]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(surprise_data, test_size=.25)

algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 33.3750


33.37496155069972

In [0]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(surprise_data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [0]:
%%time

# User product embeddings

user_product = data.groupby(['user_id', 'product_id']).agg({'product_id': 'count'}).rename(columns={'product_id': 'total_products'})
np_array_user_product = user_product.unstack('product_id').values
np_array_user_product = np.nan_to_num(np_array_user_product).astype(int)

In [0]:
np_array_user_product